# Setup
Run below for setup and data loading:

In [1]:
if(!require(lme4)){
    install.packages("lme4")
}

library(lme4) 
library(tidyr)  
library(dplyr)
library(ggplot2)

results = read.csv("model_input.csv", header=TRUE, sep=",")

Loading required package: lme4
Loading required package: Matrix

Attaching package: ‘tidyr’

The following objects are masked from ‘package:Matrix’:

    expand, pack, unpack


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
nrow(results)

[1] 1760

In [0]:
rearranged_results <- droplevels(results)

In [0]:
data <- mutate(rearranged_results, 'participant' = as.factor(participant)) %>% 
        mutate('slide' = as.factor(slide))  %>% 
        mutate('order' = as.factor(order))   %>% 
        mutate('type' = as.factor(type))  %>% 
        mutate('grade' = as.factor(grade)) %>% 
        mutate('mode' = as.factor(mode)) 

In [0]:
run_model <- function(data) {
  model0 <- glmer(accuracy ~  (1|participant) + (1|slide) + grade + type, family = 'binomial', data=data)
  model1 <- glmer(accuracy ~  (1|participant) + (1|slide) + grade + type + mode, family = 'binomial', data=data)
  print(model1)
  print(anova(model0, model1))
  se <- sqrt(diag(vcov(model1)))
  # table of estimates with 95% CI
  tab <- cbind(Est = fixef(model1), LL = fixef(model1) - 1.96 * se, UL = fixef(model1) + 1.96 * se)
  print("Odds Ratio")
  print(exp(tab))
}

# Effect of Assistance on ALL Participants

In [6]:
m = run_model(data)

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.108416 (tol = 0.001, component 1)”

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: accuracy ~ (1 | participant) + (1 | slide) + grade + type + mode
   Data: data
      AIC       BIC    logLik  deviance  df.resid 
 897.2970  946.5547 -439.6485  879.2970      1751 
Random effects:
 Groups      Name        Std.Dev.
 slide       (Intercept) 1.3449  
 participant (Intercept) 0.6191  
Number of obs: 1760, groups:  slide, 80; participant, 22
Fixed Effects:
(Intercept)       grade2       grade3        type2        type3        type4  
    4.35677     -0.24347     -1.85180     -1.59005     -1.20766     -0.05191  
      mode1  
    0.24784  
convergence code 0; 1 optimizer warnings; 0 lme4 warnings 
Data: data
Models:
model0: accuracy ~ (1 | participant) + (1 | slide) + grade + type
model1: accuracy ~ (1 | participant) + (1 | slide) + grade + type + mode
       Df    AIC    BIC  logLik deviance  Chisq Chi Df Pr(>Chisq)
model0  8 897.06 940.84 -440

# Effect of Assistance on all except Pathologist NOC

In [7]:
m = run_model(data   %>%  filter(type != 4))

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: accuracy ~ (1 | participant) + (1 | slide) + grade + type + mode
   Data: data
      AIC       BIC    logLik  deviance  df.resid 
 797.5641  839.7433 -390.7821  781.5641      1432 
Random effects:
 Groups      Name        Std.Dev.
 slide       (Intercept) 1.3313  
 participant (Intercept) 0.6578  
Number of obs: 1440, groups:  slide, 80; participant, 18
Fixed Effects:
(Intercept)       grade2       grade3        type2        type3        mode1  
     4.2889      -0.2535      -1.8403      -1.5967      -1.2106       0.4047  
Data: data
Models:
model0: accuracy ~ (1 | participant) + (1 | slide) + grade + type
model1: accuracy ~ (1 | participant) + (1 | slide) + grade + type + mode
       Df    AIC    BIC  logLik deviance  Chisq Chi Df Pr(>Chisq)  
model0  7 799.59 836.50 -392.80   785.59                           
model1  8 797.56 839.74 -390.78   781.56 4.03

# Effect of Experience Level

In [8]:
model0 <- glmer(accuracy ~ (1|participant) +  (1|slide) + grade + mode, family = 'binomial', data=data)
print(model0)
model1 <- glmer(accuracy ~ (1|participant) +  (1|slide) + grade + mode + type, family = 'binomial', data=data)
#print(model1)
print(anova(model0, model1))
se <- sqrt(diag(vcov(model1)))
# table of estimates with 95% CI
tab <- cbind(Est = fixef(model1), LL = fixef(model1) - 1.96 * se, UL = fixef(model1) + 1.96 * se)
print("Odds Ratio")
print(exp(tab))

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: accuracy ~ (1 | participant) + (1 | slide) + grade + mode
   Data: data
      AIC       BIC    logLik  deviance  df.resid 
 904.3368  937.1752 -446.1684  892.3368      1754 
Random effects:
 Groups      Name        Std.Dev.
 slide       (Intercept) 1.342   
 participant (Intercept) 0.914   
Number of obs: 1760, groups:  slide, 80; participant, 22
Fixed Effects:
(Intercept)       grade2       grade3        mode1  
     3.5805      -0.2623      -1.8612       0.2528  


Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.109021 (tol = 0.001, component 1)”

Data: data
Models:
model0: accuracy ~ (1 | participant) + (1 | slide) + grade + mode
model1: accuracy ~ (1 | participant) + (1 | slide) + grade + mode + type
       Df    AIC    BIC  logLik deviance Chisq Chi Df Pr(>Chisq)   
model0  6 904.34 937.18 -446.17   892.34                           
model1  9 897.30 946.55 -439.65   879.30 13.04      3   0.004551 **
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
[1] "Odds Ratio"
                   Est          LL          UL
(Intercept) 78.0277090 20.51819519 296.7280168
grade2       0.7837618  0.23905397   2.5696396
grade3       0.1569577  0.03646385   0.6756206
mode1        1.2811291  0.88162150   1.8616738
type2        0.2038944  0.08190900   0.5075500
type3        0.2988406  0.11853028   0.7534421
type4        0.9491016  0.31781128   2.8343668


# Effect of Tumor Grade

In [9]:
model0 <- glmer(accuracy ~ (1|participant) + (1|slide) + mode + type, family = 'binomial', data=data)
print(model0)
model1 <- glmer(accuracy ~ (1|participant) + (1|slide) + mode + type + grade, family = 'binomial', data=data)
print(model1)
print(anova(model0, model1))
se <- sqrt(diag(vcov(model1)))
# table of estimates with 95% CI
tab <- cbind(Est = fixef(model1), LL = fixef(model1) - 1.96 * se, UL = fixef(model1) + 1.96 * se)
print("Odds Ratio")
print(exp(tab))

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: accuracy ~ (1 | participant) + (1 | slide) + mode + type
   Data: data
      AIC       BIC    logLik  deviance  df.resid 
 902.5506  940.8621 -444.2753  888.5506      1753 
Random effects:
 Groups      Name        Std.Dev.
 slide       (Intercept) 1.4983  
 participant (Intercept) 0.6268  
Number of obs: 1760, groups:  slide, 80; participant, 22
Fixed Effects:
(Intercept)        mode1        type2        type3        type4  
    3.98297      0.25273     -1.59996     -1.22100     -0.07568  


Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.106512 (tol = 0.001, component 1)”

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: accuracy ~ (1 | participant) + (1 | slide) + mode + type + grade
   Data: data
      AIC       BIC    logLik  deviance  df.resid 
 897.2970  946.5547 -439.6485  879.2970      1751 
Random effects:
 Groups      Name        Std.Dev.
 slide       (Intercept) 1.3450  
 participant (Intercept) 0.6191  
Number of obs: 1760, groups:  slide, 80; participant, 22
Fixed Effects:
(Intercept)        mode1        type2        type3        type4       grade2  
    4.35693      0.24776     -1.59002     -1.20806     -0.05227     -0.24354  
     grade3  
   -1.85152  
convergence code 0; 1 optimizer warnings; 0 lme4 warnings 
Data: data
Models:
model0: accuracy ~ (1 | participant) + (1 | slide) + mode + type
model1: accuracy ~ (1 | participant) + (1 | slide) + mode + type + grade
       Df    AIC    BIC  logLik deviance  Chisq Chi Df Pr(>Chisq)   
model0  7 902.55 940.86 -4

# Effect of model correctness on accuracy

In [0]:
model_wrong_set = data %>% filter(errormodel == 1)
model_right_set = data %>% filter(errormodel == 0)


In [11]:
nrow(model_wrong_set)

[1] 278

In [12]:
nrow(model_right_set)

[1] 1482

In [13]:
model0 <- glmer(accuracy ~ (1|participant) + (1|slide) + type + grade, family = 'binomial', data=model_right_set)
model1 <- glmer(accuracy ~ (1|participant) + (1|slide) + type + grade + mode, family = 'binomial', data=model_right_set)
print(anova(model0, model1))
se <- sqrt(diag(vcov(model1)))
# table of estimates with 95% CI
tab <- cbind(Est = fixef(model1), LL = fixef(model1) - 1.96 * se, UL = fixef(model1) + 1.96 * se)
print("Odds Ratio")
print(exp(tab))

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.0189273 (tol = 0.001, component 1)”

Data: model_right_set
Models:
model0: accuracy ~ (1 | participant) + (1 | slide) + type + grade
model1: accuracy ~ (1 | participant) + (1 | slide) + type + grade + mode
       Df    AIC    BIC  logLik deviance  Chisq Chi Df Pr(>Chisq)    
model0  8 540.84 583.25 -262.42   524.84                             
model1  9 515.49 563.20 -248.75   497.49 27.346      1  1.701e-07 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
[1] "Odds Ratio"
                   Est          LL          UL
(Intercept) 67.3791161 16.98007710 267.3689440
type2        0.1656495  0.05854009   0.4687345
type3        0.2551745  0.08928931   0.7292475
type4        0.8383008  0.22650857   3.1025239
grade2       1.0097527  0.33026518   3.0872178
grade3       0.2949566  0.07200173   1.2082961
mode1        4.2892450  2.36056583   7.7937342


In [14]:
model0 <- glmer(accuracy ~ (1|participant) + (1|slide) + type + grade, family = 'binomial', data=model_wrong_set)
model1 <- glmer(accuracy ~ (1|participant) + (1|slide) + type + grade + mode, family = 'binomial', data=model_wrong_set)
print(anova(model0, model1))
se <- sqrt(diag(vcov(model1)))
# table of estimates with 95% CI
tab <- cbind(Est = fixef(model1), LL = fixef(model1) - 1.96 * se, UL = fixef(model1) + 1.96 * se)
print("Odds Ratio")
print(exp(tab))

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.00362113 (tol = 0.001, component 1)”

Data: model_wrong_set
Models:
model0: accuracy ~ (1 | participant) + (1 | slide) + type + grade
model1: accuracy ~ (1 | participant) + (1 | slide) + type + grade + mode
       Df    AIC    BIC  logLik deviance  Chisq Chi Df Pr(>Chisq)    
model0  8 311.86 340.88 -147.93   295.86                             
model1  9 296.75 329.40 -139.37   278.75 17.109      1   3.53e-05 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
[1] "Odds Ratio"
                    Est         LL          UL
(Intercept) 32.54572232 3.59929566 294.2864776
type2        0.34188852 0.07855806   1.4879155
type3        0.41450764 0.09633825   1.7834720
type4        1.46959323 0.26856089   8.0417676
grade2       0.66760067 0.09953444   4.4777532
grade3       0.08051055 0.00898094   0.7217451
mode1        0.25265215 0.12588259   0.5070845
